In [ ]:
# https://ourworldindata.org/grapher/gdp-world-regions-stacked-area
import pandas as pd

df_pib = pd.read_csv("gdp-world-regions-stacked-area.csv")
df_pib

In [ ]:
# https://data.worldbank.org/indicator/NY.GDP.MKTP.KD
import csv
import io
import re
from zipfile import ZipFile

import pandas as pd


def camel_case(words):
    return "".join(word[0].upper() + word[1:].lower() for word in words.split(" "))


with ZipFile("API_NY.GDP.MKTP.KD_DS2_en_csv_v2_3630701.zip") as myzip:
    with myzip.open("API_NY.GDP.MKTP.KD_DS2_en_csv_v2_3630701.csv") as csvfile:
        spamreader = csv.reader(
            io.TextIOWrapper(csvfile, encoding=None), delimiter=",", quotechar='"'
        )
        table = list(spamreader)
        year_re = re.compile(r"^[0-9]{4}$")
        headers = [
            (e[0], camel_case(e[1]))
            for e in enumerate(table[4])
            if e[1] and year_re.match(e[1]) is None
        ]
        years = [
            (e[0], int(e[1]))
            for e in enumerate(table[4])
            if year_re.match(e[1]) is not None
        ]

        data = {"Year": [], "GDP": []}
        for y, header in headers:
            data[header] = []
        for row in table[5:]:
            data_headers = {}
            data_headers2 = {}
            for y, header in headers:
                data_headers[header] = row[y]
                data_headers2[header] = [row[y]]

            for y, year in years:
                value = row[y]
                if value:
                    for y2, header in headers:
                        data[header].append(row[y2])
                    data["Year"].append(year)
                    data["GDP"].append(float(row[y]))

        df_pib = pd.DataFrame(data)

df_pib

In [ ]:
df_pib_sum = df_pib.groupby("Year").sum().reset_index()
df_pib_world = df_pib.query("CountryName in ['World', 'Switzerland']")
df_pib_world

In [ ]:
import plotly.express as px

fig = px.line(df_pib_world, x="Year", y="GDP", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
# https://ourworldindata.org/grapher/population-since-1800
import pandas as pd

df_population = pd.read_csv("population-since-1800.csv")
df_population

In [ ]:
df_population_world = df_population.query("Entity == ['World', 'Switzerland']")
df_population_world

In [ ]:
import plotly.express as px

fig = px.line(
    df_population_world,
    x="Year",
    y="Population (historical estimates)",
    width=800,
    height=800,
)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("bp-stats-review-2021-all-data.xlsx")

In [ ]:
for type_ in shifter_ds.metadata():
    print(f"{type_['label']} [{type_['unit']}]")

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World"],
    types_filter=[
        "Primary Energy Consumption",
        "CO2 From Flaring",
        "CO2 Excluding Flaring",
    ],
)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="Year", y="Value", color="TypeUnit", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
df_co2 = (
    df.query("Type in ['CO2 From Flaring', 'CO2 Excluding Flaring']")
    .groupby(["Year"])
    .sum()
    .reset_index()
)
df_co2.rename(columns={"Value": "CO2"}, inplace=True)
df_co2

In [ ]:
import plotly.express as px

fig = px.line(df_co2, x="Year", y="CO2", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={"rangeslider": {"visible": True}},
)
fig.show()

In [ ]:
# df_co2_pib = pd.concat([df_co2,
#    df_pib_world], axis=1

df_co2_pib = pd.merge(
    df_co2,
    df_pib_world,
    how="inner",
    on="Year",
)
df_co2_pib

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_co2_pib,
    x="CO2",
    y="GDP",
    width=800,
    height=800,
    trendline="lowess",
    hover_data=["CO2", "GDP", "Year"],
)
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_co2_pib,
    x="CO2",
    y="GDP",
    width=800,
    height=800,
    trendline="ols",
    hover_data=["CO2", "GDP", "Year"],
)
fig.update_traces(mode="lines")
fig.show()

In [ ]:
df_e_pib = pd.merge(
    df.query('Type == "Primary Energy Consumption"'),
    df_pib_world,
    how="inner",
    on="Year",
)
df_e_pib.rename(columns={"Value": "PrimaryEnergyConsumption"}, inplace=True)
df_e_pib

In [ ]:
import plotly.express as px

fig = px.scatter(
    df_e_pib,
    x="PrimaryEnergyConsumption",
    y="GDP",
    width=800,
    height=800,
    trendline="lowess",
    hover_data=["PrimaryEnergyConsumption", "GDP", "Year"],
)
fig.update_layout()
fig.show()

In [ ]:
df_co2_pib2 = df_co2_pib.assign(CO2_P_PIB=df_co2_pib.CO2 / df_co2_pib.GDP)
df_co2_pib2

In [ ]:
import plotly.express as px

fig = px.line(
    df_co2_pib2, x="Year", y="CO2_P_PIB", width=800, height=800, title="Efficacité"
)
fig.update_layout()
fig.show()